Cleaning and Analyzing Vital Signs Data (EHR-style)

Use case:
You receive blood pressure and heart rate data exported from an EHR as a CSV file.

Example: Load, clean, and summarize vitals

In [2]:
import numpy as np
import pandas as pd

np.random.seed(7)

n = 250
df = pd.DataFrame({
    "patient_id": np.random.randint(1000, 1100, size=n),
    "encounter_date": pd.to_datetime("2025-10-01") + pd.to_timedelta(np.random.randint(0, 60, size=n), unit="D"),
    "age": np.random.randint(18, 90, size=n),
    "sex": np.random.choice(["F", "M"], size=n),
    "systolic": np.random.normal(132, 18, size=n).round(0),
    "diastolic": np.random.normal(82, 12, size=n).round(0),
    "heart_rate": np.random.normal(78, 12, size=n).round(0)
})

# Inject missingness (common in EHR exports)
mask = np.random.rand(n) < 0.06
df.loc[mask, "systolic"] = np.nan
mask = np.random.rand(n) < 0.05
df.loc[mask, "diastolic"] = np.nan

df.head()


,patient_id,encounter_date,age,sex,systolic,diastolic,heart_rate
0,1047,2025-11-06,22,F,150.0,93.0,93.0
1,1068,2025-10-06,18,F,123.0,71.0,98.0
2,1025,2025-10-29,70,F,106.0,55.0,70.0
3,1067,2025-11-13,38,M,145.0,75.0,78.0
4,1083,2025-10-30,49,F,152.0,87.0,85.0


In [3]:
df.info()

missing = df.isna().mean().sort_values(ascending=False)
missing


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 250 entries, 0 to 249
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   patient_id      250 non-null    int64         
 1   encounter_date  250 non-null    datetime64[ns]
 2   age             250 non-null    int64         
 3   sex             250 non-null    object        
 4   systolic        240 non-null    float64       
 5   diastolic       238 non-null    float64       
 6   heart_rate      250 non-null    float64       
dtypes: datetime64[ns](1), float64(3), int64(2), object(1)
memory usage: 13.8+ KB


diastolic         0.048
systolic          0.040
patient_id        0.000
age               0.000
encounter_date    0.000
sex               0.000
heart_rate        0.000
dtype: float64

In [4]:
clean = df.dropna(subset=["systolic", "diastolic"]).copy()

# Plausible physiological ranges (example rules)
clean = clean[
    clean["systolic"].between(70, 260) &
    clean["diastolic"].between(40, 160) &
    clean["heart_rate"].between(30, 220)
].copy()

clean.shape


(230, 7)

In [5]:
def bp_category(sys, dia):
    if sys < 120 and dia < 80:
        return "Normal"
    elif 120 <= sys < 130 and dia < 80:
        return "Elevated"
    elif (130 <= sys < 140) or (80 <= dia < 90):
        return "Hypertension Stage 1"
    else:
        return "Hypertension Stage 2"

clean["bp_category"] = [bp_category(s, d) for s, d in zip(clean["systolic"], clean["diastolic"])]
clean["bp_category"].value_counts()


bp_category
Hypertension Stage 1    112
Hypertension Stage 2     79
Normal                   23
Elevated                 16
Name: count, dtype: int64

In [6]:
bp_control = ((clean["systolic"] < 140) & (clean["diastolic"] < 90)).mean()
print(f"BP control rate (<140/90): {bp_control*100:.1f}%")

# Breakdown by sex
by_sex = clean.assign(control=((clean["systolic"] < 140) & (clean["diastolic"] < 90))).groupby("sex")["control"].mean()
(by_sex * 100).round(1)


BP control rate (<140/90): 47.4%


sex
F    47.5
M    47.3
Name: control, dtype: float64

In [7]:
clean["high_risk_flag"] = (
    (clean["bp_category"] == "Hypertension Stage 2") |
    (clean["heart_rate"] >= 100) |
    (clean["age"] >= 75)
)

clean["high_risk_flag"].value_counts()
clean.loc[clean["high_risk_flag"]].head(10)


,patient_id,encounter_date,age,sex,systolic,diastolic,heart_rate,bp_category,high_risk_flag
0,1047,2025-11-06,22,F,150.0,93.0,93.0,Hypertension Stage 2,True
3,1067,2025-11-13,38,M,145.0,75.0,78.0,Hypertension Stage 2,True
5,1023,2025-11-19,67,M,129.0,107.0,72.0,Hypertension Stage 2,True
8,1014,2025-11-22,82,F,159.0,107.0,89.0,Hypertension Stage 2,True
10,1072,2025-11-01,38,F,141.0,79.0,92.0,Hypertension Stage 2,True
13,1090,2025-11-17,18,M,141.0,100.0,97.0,Hypertension Stage 2,True
16,1068,2025-11-18,46,F,156.0,76.0,83.0,Hypertension Stage 2,True
19,1044,2025-10-14,63,F,100.0,90.0,77.0,Hypertension Stage 2,True
20,1000,2025-10-20,78,F,109.0,85.0,80.0,Hypertension Stage 1,True
23,1006,2025-10-25,40,F,128.0,90.0,85.0,Hypertension Stage 2,True


In [8]:
import numpy as np
import pandas as pd

np.random.seed(11)

patients = [201, 202, 203, 204]
days = pd.date_range("2025-11-01", periods=30, freq="D")

rpm = []
for pid in patients:
    base_sys = np.random.normal(130, 8)
    base_dia = np.random.normal(82, 5)
    for d in days:
        # Simulate occasional missing readings (adherence gaps)
        if np.random.rand() < 0.12:
            continue
        rpm.append({
            "patient_id": pid,
            "date": d,
            "systolic": np.random.normal(base_sys, 10),
            "diastolic": np.random.normal(base_dia, 6),
            "heart_rate": np.random.normal(76, 8)
        })

rpm = pd.DataFrame(rpm)
rpm["systolic"] = rpm["systolic"].round(0)
rpm["diastolic"] = rpm["diastolic"].round(0)
rpm["heart_rate"] = rpm["heart_rate"].round(0)

rpm.head()


,patient_id,date,systolic,diastolic,heart_rate
0,201,2025-11-01,141.0,81.0,82.0
1,201,2025-11-02,144.0,77.0,79.0
2,201,2025-11-03,142.0,82.0,75.0
3,201,2025-11-05,144.0,83.0,71.0
4,201,2025-11-06,131.0,74.0,82.0


In [9]:
expected_days = len(days)
adherence = rpm.groupby("patient_id")["date"].nunique().to_frame("days_with_reading")
adherence["expected_days"] = expected_days
adherence["adherence_rate"] = adherence["days_with_reading"] / adherence["expected_days"]
(adherence["adherence_rate"] * 100).round(1)


patient_id
201    83.3
202    93.3
203    80.0
204    83.3
Name: adherence_rate, dtype: float64

In [10]:
rpm_sorted = rpm.sort_values(["patient_id", "date"]).copy()
rpm_sorted["sys_7day_avg"] = rpm_sorted.groupby("patient_id")["systolic"].transform(lambda s: s.rolling(7, min_periods=3).mean())
rpm_sorted["dia_7day_avg"] = rpm_sorted.groupby("patient_id")["diastolic"].transform(lambda s: s.rolling(7, min_periods=3).mean())

rpm_sorted.head(10)


,patient_id,date,systolic,diastolic,heart_rate,sys_7day_avg,dia_7day_avg
0,201,2025-11-01,141.0,81.0,82.0,NaN,NaN
1,201,2025-11-02,144.0,77.0,79.0,NaN,NaN
2,201,2025-11-03,142.0,82.0,75.0,142.333333,80.000000
3,201,2025-11-05,144.0,83.0,71.0,142.750000,80.750000
4,201,2025-11-06,131.0,74.0,82.0,140.400000,79.400000
5,201,2025-11-08,160.0,87.0,74.0,143.666667,80.666667
6,201,2025-11-09,155.0,94.0,66.0,145.285714,82.571429
7,201,2025-11-10,137.0,80.0,71.0,144.714286,82.428571
8,201,2025-11-12,142.0,78.0,77.0,144.428571,82.571429
9,201,2025-11-13,145.0,93.0,85.0,144.857143,84.142857


In [11]:
# Alert if 7-day avg systolic >= 140 OR diastolic >= 90
rpm_sorted["alert_flag"] = (rpm_sorted["sys_7day_avg"] >= 140) | (rpm_sorted["dia_7day_avg"] >= 90)

alerts = rpm_sorted[rpm_sorted["alert_flag"]].groupby("patient_id").size().rename("alert_days")
alerts


patient_id
201    23
Name: alert_days, dtype: int64